# Optimization !  
Looking for the best route with optuna.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from stravart.utils import *
from stravart.polygone import *
from stravart.operations import *
from stravart.visualization import *
from stravart.operations import *
from stravart.metrics import *

import optuna

cat_face_coordinates = [
    # Start from the bottom of the left ear
    (-4, 7), (-3.5, 8), (-3, 9), (-2.5, 10), (-2, 10.5), (-1.5, 11), (-1, 11.5), (-0.5, 11.75), (0, 12),
    # Top of head to the right ear
    (0.5, 11.75), (1, 11.5), (1.5, 11), (2, 10.5), (2.5, 10), (3, 9), (3.5, 8), (4, 7),
    # Right side of the face
    (3.5, 6), (3, 5), (2.5, 4), (2, 3), (1.5, 2), (1, 1), (0.5, 0), (0, -1),
    # Bottom of the face and left side
    (-0.5, 0), (-1, 1), (-1.5, 2), (-2, 3), (-2.5, 4), (-3, 5), (-3.5, 6), (-4, 7),
    # Adding whiskers on the left side
    (-4.5, 6.5), (-5, 6), (-5.5, 5.5), (-6, 5), (-5.5, 4.5), (-5, 4), (-4.5, 3.5), (-4, 3),
    # Returning to the bottom of the left ear
    (-3.5, 3.5), (-3, 4), (-2.5, 4.5), (-2, 5), (-1.5, 5.5), (-1, 6), (-0.5, 6.5), (0, 7),
    # Crossing to the right side
    (0.5, 6.5), (1, 6), (1.5, 5.5), (2, 5), (2.5, 4.5), (3, 4), (3.5, 3.5), (4, 3),
    # Adding whiskers on the right side
    (4.5, 3.5), (5, 4), (5.5, 4.5), (6, 5), (5.5, 5.5), (5, 6), (4.5, 6.5), (4, 7),
    # Completing the loop back to the start
    (3.5, 7.5), (3, 8), (2.5, 8.5), (2, 9), (1.5, 9.5), (1, 10), (0.5, 10.5), (0, 11),
    (-0.5, 10.5), (-1, 10), (-1.5, 9.5), (-2, 9), (-2.5, 8.5), (-3, 8), (-3.5, 7.5), (-4, 7)
]

origin = simplify_coordinates(cat_face_coordinates)
poly =  Polygon.from_list(coordinates_list=origin, system="cartesian")
normed_poly = poly.scale_coordinates()

In [ ]:
def generate_grid(lat_start, lat_end, lon_start, lon_end, lat_points, lon_points):
    lat_step = (lat_end - lat_start) / (lat_points - 1)
    lon_step = (lon_end - lon_start) / (lon_points - 1)

    grid = []
    for i in range(lat_points):
        for j in range(lon_points):
            lat = lat_start + i * lat_step
            lon = lon_start + j * lon_step
            grid.append((lat, lon))
    
    return grid

def generate_route(gps_poly):
    bicycle_contour = gps_poly.get_nearest_bicycle_road_points(dist=2000)
    final_contour, path_mapping = Route(bicycle_contour).fill_paths_between_points()
    return final_contour, path_mapping

def define_search_space(trial, city_grid):
    angle = trial.suggest_float('rot_angle', -20, 20, step=5)

    # Randomly select a map center from city_grid
    map_center_idx = trial.suggest_int('map_center_idx', 0, len(city_grid) - 1)
    map_center = city_grid[map_center_idx]

    # Use discrete increments for radius
    #radius = trial.suggest_discrete_uniform('radius', 0.01, 0.1, 0.005)
    radius = trial.suggest_float('radius', 0.01, 0.1, step= 0.005)

    return angle, map_center, radius

def objective(trial, poly, city_grid):
    angle, map_center, radius = define_search_space(trial,city_grid=city_grid)

    # Apply the operation and projection
    new_poly = Rotation(angle).apply(poly)
    projection = Projection(center=map_center, radius=radius, map_type="GPS")
    gps_poly = projection.apply(new_poly)

    # Generate route and calculate loss
    final_contour, path_mapping = generate_route(gps_poly)
    loss = diff_area(final_contour, path_mapping)

    trial.set_user_attr('final_contour', final_contour)
    trial.set_user_attr('path_mapping', path_mapping)

    return loss

In [ ]:
# Define bounding box of Paris
lat_start, lat_end = 48.8156, 48.9022
lon_start, lon_end = 2.2241, 2.4699

city_grid = generate_grid(lat_start, lat_end, lon_start, lon_end, 5, 5)

study = optuna.create_study(direction='minimize')
#study.optimize(objective, n_trials=5)
study.optimize(lambda trial: objective(trial, poly=poly, city_grid=city_grid), n_trials=10)

# Print the best parameters
print("Best trial:")
best_trial = study.best_trial
best_final_contour = best_trial.user_attrs['final_contour']
best_path_mapping = best_trial.user_attrs['path_mapping']
map_center = city_grid[study.best_params['map_center_idx']]

poly_final = Polygon.from_route(best_final_contour, system="GPS")

print(" Value of loss:", best_trial.value)
print(" Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

print("Route length (kms): ", poly_final.perimeter)

In [ ]:
plot_route(map_center=map_center,route=best_final_contour, points=True)